# **Laboratorio 8**
- Joaquín Campos - 22155
- Sofía García - 22210
- Julio García Salas - 22076

## **Inciso 1**

In [0]:
# INCISO 1 — Volumes (como tu lab) + fallback pandas -> Spark (estable en Spark Connect)
PATH = "/Volumes/workspace/default/lab8/_UNIFICADO.csv"

from pyspark.sql import functions as F
import pandas as pd
import csv, io

# 0) Ver que exista el archivo y un vistazo rápido (igual a tu patrón)
parent = "/Volumes/workspace/default/lab8"
display(dbutils.fs.ls(parent))

print("=== _UNIFICADO.csv (inicio) ===")
raw = dbutils.fs.head(PATH, 2048)
print(raw[:1000])

# 1) Cargar con pandas (robusto a BOM, separador, y Volumes) y pasar a Spark
#    - dtype=str para no perder ceros ni convertir 9 -> 9.0
#    - encoding utf-8-sig para limpiar BOM en el header
pdf = pd.read_csv(PATH, dtype=str, encoding="utf-8-sig", engine="python")

# Limpieza suave: strip espacios en nombres de columnas
pdf.columns = [c.strip() for c in pdf.columns]

# Crear DataFrame Spark desde pandas
df_all = spark.createDataFrame(pdf)

print(f"\nTotal filas en _UNIFICADO: {df_all.count():,}")
print(f"Total columnas: {len(df_all.columns)}")
print("Columnas (primeras 12):", df_all.columns[:12])

# 2) Heurísticas para separar 'tablas'
hechos_df = df_all.filter(F.col("tipo_eve").isNotNull())
vehiculos_df = df_all.filter(
    F.col("tipo_veh").isNotNull() |
    F.col("marca_veh").isNotNull() |
    F.col("modelo_veh").isNotNull() |
    F.col("color_veh").isNotNull()
)
personas_df = df_all.filter(
    F.col("sexo_per").isNotNull() |
    F.col("edad_per").isNotNull() |
    F.col("sexo_con").isNotNull() |
    F.col("edad_con").isNotNull()
)

print("\n=== Conteos por 'tabla' (según heurística) ===")
print(f"Hechos (tipo_eve no nulo): {hechos_df.count():,}")
print(f"Vehículos (campos de vehículo no nulos): {vehiculos_df.count():,}")
print(f"Personas (campos de personas/conductores no nulos): {personas_df.count():,}")

# 3) Muestras
print("\n--- Muestra HECHOS ---")
hechos_df.select(*[c for c in ["anio_ocu","mes_ocu","dia_ocu","dia_sem_ocu","hora_ocu","depto_ocu","muni_ocu","tipo_eve","zona_ocu"] if c in hechos_df.columns]).show(10, truncate=False)

print("\n--- Muestra VEHÍCULOS ---")
vehiculos_df.select(*[c for c in ["anio_ocu","mes_ocu","hora_ocu","tipo_veh","marca_veh","modelo_veh","color_veh","g_modelo_veh"] if c in vehiculos_df.columns]).show(10, truncate=False)

print("\n--- Muestra PERSONAS ---")
personas_df.select(*[c for c in ["anio_ocu","mes_ocu","hora_ocu","sexo_per","edad_per","sexo_con","edad_con","mayor_menor","g_edad","g_edad_2","g_edad_60ymas","g_edad_80ymas","edad_quinquenales"] if c in personas_df.columns]).show(10, truncate=False)

# 4) Describe/Summary (si hay filas)
def safe_describe(df, cols, title):
    cols_exist = [c for c in cols if c in df.columns]
    if cols_exist and df.count() > 0:
        print(f"\n--- DESCRIBE: {title} ---")
        df.select(*cols_exist).describe().show(truncate=False)
        print(f"\n--- SUMMARY: {title} ---")
        df.select(*cols_exist).summary().show(truncate=False)
    else:
        print(f"\n[INFO] Sin filas o columnas para {title}")

safe_describe(hechos_df, ["anio_ocu","mes_ocu","dia_ocu","hora_ocu","depto_ocu","muni_ocu"], "HECHOS (tiempo y ubicación)")
safe_describe(vehiculos_df, ["anio_ocu","tipo_veh","marca_veh","modelo_veh","color_veh","g_modelo_veh"], "VEHÍCULOS (características)")
safe_describe(personas_df, ["anio_ocu","sexo_per","edad_per","sexo_con","edad_con","mayor_menor","g_edad","g_edad_2"], "PERSONAS (atributos)")

# 5) Vistas temporales para los incisos siguientes
df_all.createOrReplaceTempView("accidentes_unificado")
hechos_df.createOrReplaceTempView("hechos")
vehiculos_df.createOrReplaceTempView("vehiculos")
personas_df.createOrReplaceTempView("personas")


path,name,size,modificationTime
dbfs:/Volumes/workspace/default/lab8/_UNIFICADO.csv,_UNIFICADO.csv,10395125,1759426297000


=== _UNIFICADO.csv (inicio) ===
[Truncated to first 2048 bytes]
﻿anio_ocu,mes_ocu,dia_ocu,dia_sem_ocu,hora_ocu,g_hora,g_hora_5,depto_ocu,muni_ocu,area_geo_ocu,zona_ocu,zona_ciudad,num_corre,tipo_eve,tipo_veh,marca_veh,color_veh,modelo_veh,g_modelo_veh,sexo_pil,edad_pil,g_edad_pil,sexo_con,edad_con,sexo_per,edad_per,g_edad,g_edad_2,g_edad_60ymas,g_edad_80ymas,edad_quinquenales,mayor_menor,estado_pil,estado_con,condicion_pil,causa_acc,causa_ac,color_v,edad_m1,modelo_v,tipo_v,source_file
2009,5.0,13.0,3.0,1.0,,,14.0,,1.0,,,1449.0,,3.0,,2.0,1988.0,,9.0,999.0,99.0,,,,,,,,,,,2.0,,,2.0,,,,,,2009.csv
2009,7.0,1.0,1.0,15.0,,,22.0,,2.0,,,1587.0,,4.0,,4.0,9999.0,,2.0,13.0,2.0,,,,,,,,,,,1.0,,,2.0,,,,,,2009.csv
2009,3.0,1.0,7.0,17.0,,,17.0,,2.0,,,679.0,,4.0,,99.0,9999.0,,1.0,14.0,2.0,,,,,,,,,,,1.0,,,4.0,,,,,,2009.csv
2009,2.0,26.0,4.0,18.0,,,21.0,,2.0,,,633.0,,1.0,,1.0,9999.0,,1.0,15.0,2.0,,,,,,,,,,,9.0,,,4.0,,,,,,2009.csv
2009,3.0,1.0,7.0,15.0,,,10.0,,1.0,,,673.0,,14.0,,1.0,2007.0,,1.0,15.0,2.0,,,

# Análisis exploratorio del archivo `_UNIFICADO.csv`

## 1) Visión general
- **Filas totales:** 89,317  
- **Columnas:** 42  
- **Primeras columnas:** `anio_ocu, mes_ocu, dia_ocu, dia_sem_ocu, hora_ocu, g_hora, g_hora_5, depto_ocu, muni_ocu, area_geo_ocu, zona_ocu, zona_ciudad`
- El encabezado muestra variables de **tiempo** (año, mes, día, hora), **ubicación** (depto/muni, zona), **tipología** (`tipo_eve`, `tipo_veh`), **vehículo** (marca, modelo, color) y **personas** (sexo/edad de piloto, conductor y persona).

## 2) Segmentación heurística (tablas lógicas)
- **Hechos de tránsito (tipo_eve no nulo):** 70,435  
- **Vehículos (campos de vehículo no nulos):** 86,130  
- **Personas (campos de personas/conductores no nulos):** 12,505  

La cantidad de **vehículos > hechos** es consistente con la posibilidad de múltiples vehículos por evento. La tabla de **personas** es la más pequeña.

## 3) Muestras y primeras impresiones
- En **Hechos**, las filas de ejemplo incluyen `anio_ocu=2014`, con hora y claves de ubicación pobladas; se observan **valores repetidos** en los primeros registros mostrados, lo cual sugiere posibles duplicados.
- En **Vehículos**, aparecen patrones de **códigos sentinela** como `modelo_veh=9999`, `color_veh=99` y `tipo_veh` con valores numéricos y también literales (p. ej., *Tractor*). Hay marcas específicas (p. ej., *Zongshen*).
- En **Personas**, predominan registros con edad del conductor (`edad_con`) y **muchos nulos** en `sexo_per`/`edad_per`, lo que sugiere que esta subtabla no siempre está completa para todas las víctimas/personas.

## 4) Métricas descriptivas destacadas

### Hechos (tiempo y ubicación)
- **Rango de años:** 2014 — 2023  
- **Mediana de hora:** ~15 (3 p. m.), con **media** ≈ 13.66; sugiere concentración en horas diurnas.  
- **Inconsistencias tipológicas:**  
  - `mes_ocu` contiene valores **textuales** como *Septiembre*, mezclados con numéricos.  
  - `hora_ocu` muestra valores textuales como *Ignorada*.  
  - `depto_ocu` y `muni_ocu` incluyen **nombres** (*Zacapa*, *Zunilito*) además de **códigos** numéricos.  
Estas señales revelan **heterogeneidad de formatos** (códigos vs. nombres) y **valores no numéricos** en campos esperados como numéricos.

### Vehículos (características)
- **Rango de años:** 2009 — 2023 (más amplio que “Hechos”).  
- **Modelo de vehículo (`modelo_veh`):** presencia de `9999` y literal *Ignorado* en `g_modelo_veh`, lo que sugiere codificación de **desconocido/ignorados**.  
- **Tipo de vehículo (`tipo_veh`):** mezcla de códigos numéricos y categorías textuales (p. ej., *Tractor*).  
- **Marca (`marca_veh`):** valores variados; aparecen marcas poco comunes (p. ej., *Zongshen*).

### Personas (atributos)
- **Rango de años:** 2014 — 2015 (acotado).  
- **Cobertura incompleta:** ~6.8k registros con `sexo_per`/`edad_per` frente a 12.5k totales; en `sexo_con`/`edad_con` hay 5.6k.  
- **Codificación de rangos/agrupaciones:** variables como `mayor_menor`, `g_edad`, `g_edad_60ymas`, `g_edad_80ymas` sugieren **binnings** o agrupaciones discretas.  
- Presencia de **códigos sentinela** (`999`, `9`) para desconocidos/ignorados.

## 5) Calidad de datos y codificaciones
- **Sentinelas / Desconocidos:** `99`, `999`, `9999`, y literales como *Ignorada*/*Ignorado*.  
- **Tipos mezclados:** columnas de tiempo/ubicación (mes, hora, depto, muni) combinan **numéricos y texto**, lo que impedirá agregaciones sin **normalización previa**.  
- **Años por subtabla:**  
  - Hechos: 2014–2023  
  - Vehículos: 2009–2023  
  - Personas: 2014–2015  
  Esto sugiere que el archivo unificado consolida series con coberturas temporales distintas; afectará cruces y comparabilidad por año.
- **Posibles duplicados:** la muestra de “Hechos” repite varias filas idénticas al inicio; conviene verificar unicidad con una llave compuesta (año, mes, día, hora, depto, muni, tipo de evento, y correlativos si existen).

## 8) Conclusión
El archivo unificado integra múltiples subfuentes con distinta **cobertura temporal** y **esquemas parcialmente heterogéneos**. Antes de responder preguntas analíticas (incisos 5–17), conviene realizar una **normalización mínima** (mes/hora/ubicación, sentinelas y duplicados). Tras esa limpieza, los conteos reportados (hechos: 70,435; vehículos: 86,130; personas: 12,505) y los rangos temporales detectados permitirán avanzar con agregaciones y uniones de forma consistente.


## **Inciso 2**

In [0]:
# INCISO 2 — Años disponibles en cada tabla + validación de coincidencia
# Requiere que existan los DataFrames: df_all, hechos_df, vehiculos_df, personas_df (del Inciso 1)

from pyspark.sql import functions as F

def years_summary(df, name):
    # detectar nombre de columna de año
    year_col = "anio_ocu" if "anio_ocu" in df.columns else ("año_ocu" if "año_ocu" in df.columns else None)
    if not year_col:
        print(f"[WARN] {name}: no existe columna 'anio_ocu'/'año_ocu'")
        return set()

    # extraer 4 dígitos consecutivos como año y castear a int (filtrar rango razonable)
    df_norm = (
        df.withColumn("anio_norm_raw", F.col(year_col).cast("string"))
          .withColumn("anio_norm", F.regexp_extract(F.col("anio_norm_raw"), r"(\d{4})", 1).cast("int"))
          .where(F.col("anio_norm").between(1990, 2035))
    )

    # conjunto de años ordenado
    years = [r["anio_norm"] for r in df_norm.select("anio_norm").distinct().orderBy("anio_norm").collect()]

    print(f"\n{name}: años (normalizados): {years}")
    print(f"{name}: registros con año válido: {df_norm.count():,} / total: {df.count():,}")

    # tabla de apoyo para inspección
    display(df_norm.groupBy("anio_norm").count().orderBy("anio_norm"))

    return set(years)

# --- calcular años por 'tabla' lógica
hechos_years    = years_summary(hechos_df,   "Hechos")
vehiculos_years = years_summary(vehiculos_df,"Vehículos")
personas_years  = years_summary(personas_df, "Personas")

# --- validación de coincidencia
present_sets = [s for s in [hechos_years, vehiculos_years, personas_years] if s]

if present_sets:
    intersec = set.intersection(*present_sets)
    union    = set.union(*present_sets)

    print("\n=== Validación de años entre tablas ===")
    print(f"Intersección (en TODAS): {sorted(intersec)}")
    print(f"Unión (en ALGUNA):      {sorted(union)}")

    for name, s in [("Hechos", hechos_years), ("Vehículos", vehiculos_years), ("Personas", personas_years)]:
        if s:
            faltan = sorted(union - s)
            print(f"- {name}: años ausentes vs unión → {faltan}")
        else:
            print(f"- {name}: sin conjunto de años (ver aviso arriba)")
else:
    print("[WARN] No se pudieron determinar años en ninguna tabla. Verificar columnas y contenido.")



Hechos: años (normalizados): [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
Hechos: registros con año válido: 70,435 / total: 70,435


anio_norm,count
2014,5651
2015,6854
2016,7964
2017,5879
2018,6395
2019,7047
2020,6350
2021,8153
2022,7924
2023,8218



Vehículos: años (normalizados): [2009, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
Vehículos: registros con año válido: 86,130 / total: 86,130


anio_norm,count
2009,3528
2011,2693
2012,3150
2013,6324
2014,5651
2015,6854
2016,7964
2017,5879
2018,6395
2019,7047



Personas: años (normalizados): [2014, 2015]
Personas: registros con año válido: 12,505 / total: 12,505


anio_norm,count
2014,5651
2015,6854



=== Validación de años entre tablas ===
Intersección (en TODAS): [2014, 2015]
Unión (en ALGUNA):      [2009, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
- Hechos: años ausentes vs unión → [2009, 2011, 2012, 2013]
- Vehículos: años ausentes vs unión → []
- Personas: años ausentes vs unión → [2009, 2011, 2012, 2013, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]


# Inciso 2 — ¿Los años coinciden entre las tablas?

## Resumen
- **Intersección (años presentes en TODAS las tablas):** **2014–2015**
- **Unión (años presentes en AL MENOS una tabla):** **2009, 2011–2013, 2014–2023**
- **Año 2010:** **no aparece** en ninguna tabla.

## Cobertura por tabla
- **Hechos:** 2014–2023  
  - **Ausentes respecto a la unión:** 2009, 2011, 2012, 2013
- **Vehículos:** 2009, 2011–2013, 2014–2023  
  - **Ausentes respecto a la unión:** *ninguno*
- **Personas:** 2014–2015  
  - **Ausentes respecto a la unión:** 2009, 2011–2013, 2016–2023

## Consistencia interna por años clave
- En **2014** y **2015** los conteos por año coinciden entre las tres tablas (p. ej., 2014 → 5,651; 2015 → 6,854), lo que sugiere consistencia para integraciones en ese bimestre temporal.

## Implicaciones para el análisis
- **Uniones Hechos ↔ Vehículos:** viables en **2014–2023** (máxima superposición entre ambas).
- **Uniones que involucren Personas:** restringidas a **2014–2015**.
- **Análisis sólo con Vehículos:** posible en **2009, 2011–2013, 2014–2023**.
- Se recomienda **documentar** la ausencia de 2010 y la **diferente cobertura temporal** de cada subtabla antes de cálculos comparativos entre años.

## Recomendación operativa
- Normalizar el campo de año (`anio_ocu`) a entero y **filtrar por la ventana temporal adecuada** según el cruce:
  - Para Hechos↔Vehículos: 2014–2023.
  - Para Hechos↔Vehículos↔Personas: 2014–2015.


## **Inicso 3**

In [0]:
# INCISO 3 — Valores distintos de "tipo de accidente" (tipo_eve)
# Requiere hechos_df (del Inciso 1). Si no existe, se toma desde df_all.

from pyspark.sql import functions as F

# 1) Asegurar fuente
if "hechos_df" not in locals():
    hechos_df = df_all.filter(F.col("tipo_eve").isNotNull())

# 2) Normalización suave para listar valores únicos con más claridad
hechos_norm = (
    hechos_df
    .withColumn("tipo_eve_str", F.col("tipo_eve").cast("string"))
    .withColumn("tipo_eve_norm",
                F.upper(F.trim(F.col("tipo_eve_str"))))  # trim + mayúsculas
    .withColumn("tipo_eve_norm",
                F.when(F.col("tipo_eve_norm").isin("", "NULL", "N/A", "NA", "IGNORADA"), None)
                 .otherwise(F.col("tipo_eve_norm")))
)

# 3) Distintos valores (normalizados) y su frecuencia
distinct_vals = (
    hechos_norm
    .groupBy("tipo_eve_norm")
    .count()
    .orderBy(F.desc("count"), F.asc("tipo_eve_norm"))
)

print("=== Distintos 'tipo_eve' (normalizados) con conteo ===")
display(distinct_vals)

# 4) (Opcional) Vista cruda sin normalizar, por si se quiere comparar
distinct_raw = (
    hechos_df
    .groupBy("tipo_eve")
    .count()
    .orderBy(F.desc("count"), F.asc("tipo_eve"))
)

print("=== Distintos 'tipo_eve' (crudo) con conteo ===")
display(distinct_raw)

# 5) (Opcional) Presencia por año, para ver en qué años aparece cada tipo
if "anio_ocu" in hechos_df.columns:
    por_anio = (
        hechos_norm
        .where(F.col("tipo_eve_norm").isNotNull())
        .groupBy("anio_ocu", "tipo_eve_norm")
        .count()
        .orderBy("anio_ocu", F.desc("count"))
    )
    print("=== Aparición por año (tipo_eve_norm) ===")
    display(por_anio)
else:
    print("[INFO] Columna 'anio_ocu' no disponible para desglosar por año.")


=== Distintos 'tipo_eve' (normalizados) con conteo ===


tipo_eve_norm,count
1,19631
5,13178
1.0,6818
5.0,4340
COLISIÓN,4319
ATROPELLO,4260
COLISI?N,4230
2,3897
DERRAPE,1982
3,1369


=== Distintos 'tipo_eve' (crudo) con conteo ===


tipo_eve,count
1,19631
5,13178
1.0,6818
5.0,4340
Colisión,4319
Atropello,4260
Colisi?n,4230
2,3897
Derrape,1982
3,1369


=== Aparición por año (tipo_eve_norm) ===


anio_ocu,tipo_eve_norm,count
2014,1.0,2345
2014,5.0,2276
2014,2.0,532
2014,3.0,366
2014,4.0,132
2015,1,3006
2015,5,2565
2015,2,715
2015,3,438
2015,4,130


# Inciso 3 — Análisis de valores distintos en **tipo_eve**

## Resumen
- Se observan **tres representaciones** para el tipo de accidente:
  1) **Códigos numéricos** (`1, 2, 3, 4, 5, 6, 7, 8, 99`)
  2) **Códigos numéricos con punto** (`1.0, 2.0, …, 99.0`)
  3) **Etiquetas textuales** (p. ej., `COLISIÓN`, `ATROPELLO`, `DERRAPE`, `CHOQUE`, `VUELCO`, `EMBARRANCÓ`)
- Aparecen **problemas de codificación** con caracteres de acento reemplazados por `?`: `COLISI?N`, `EMBARRANC?`, `CA?DA`, `ENCUNET?`.

## Frecuencias destacadas (normalizadas)
- Códigos más frecuentes: `1` (19,631), `5` (13,178), `2` (3,897), `6` (1,041), `3` (1,369), `4` (1,248), `7` (104), `8` (10), `99` (11).
- Con sufijo `.0` (variación de formato): `1.0` (6,818), `5.0` (4,340), `2.0` (1,063), `6.0` (843), `3.0` (524), `4.0` (132), `7.0` (68), `8.0` (12), `99.0` (4).
- Etiquetas textuales (limpias y con errores de codificación):
  - Limpias: `COLISIÓN` (4,319), `ATROPELLO` (4,260), `DERRAPE` (1,982), `CHOQUE` (915), `VUELCO` (309), `EMBARRANCÓ` (70).
  - Con `?`: `COLISI?N` (4,230), `EMBARRANC?` (52), `CA?DA` (3), `ENCUNET?` (2).

## Evolución temporal de la codificación
- **2014–2020:** predominan códigos numéricos (`1, 5, 2, 3, 4, 6, 7, 8, 99`).
- **2021:** los códigos aparecen con **sufijo decimal** (`1.0, 5.0, 6.0, 2.0, 3.0, 7.0, 8.0, 99.0`), indicando una **variación de formato**.
- **2022:** transición a **etiquetas textuales**, pero con **errores de codificación** (`COLISI?N`, `EMBARRANC?`, etc.).
- **2023:** etiquetas textuales **limpias** (acentos correctos: `COLISIÓN`, `ATROPELLO`, etc.).

**Implicación:** el campo `tipo_eve` cambia de **códigos** → **códigos con decimal** → **etiquetas textuales**, con un año intermedio (2022) afectado por **encoding**. Para análisis longitudinal se requiere **estandarización**.

## Inferencia de mapeos (alta probabilidad)
A partir de conteos y patrones por año (comparando frecuencias de 2018–2021 vs. 2022–2023):

- `1` / `1.0` → **COLISIÓN**  
  (máximos relativos similares a `COLISIÓN` en 2023)
- `5` / `5.0` → **ATROPELLO**
- `2` / `2.0` → **DERRAPE**
- `6` / `6.0` → **CHOQUE**  
  (frecuencias de `6` en 2018–2021 cercanas a `CHOQUE` en 2023)
- `4` / `4.0` → **VUELCO**
- `7` / `7.0` → **EMBARRANCÓ**
- `8` / `8.0` → **otros de baja frecuencia** (candidatos: **CAÍDA**, **ENCUNETADO**; requiere verificación manual por escasez de casos)
- `99` / `99.0` → **No clasificado / Ignorado**

> Nota: `CHOQUE` y `COLISIÓN` coexisten como categorías **distintas** en las etiquetas recientes (2023), por lo que no deben fusionarse.

## Problemas de calidad detectados
- **Codificación de caracteres (encoding):** presencia de `?` en lugar de acentos (UTF-8 vs. Latin-1).  
- **Doble representación del mismo código:** entero y con `.0`.  
- **Heterogeneidad de tipos:** combinación de **numéricos** y **texto** en la misma columna a través del tiempo.

## Recomendación de normalización
1. **Armonizar códigos y texto:**
   - Convertir `1.0 → 1`, `5.0 → 5`, etc.
   - Mapear códigos ↔ etiquetas según la tabla de inferencias anterior.
2. **Corregir encoding en la etapa textual (2022):**
   - Reemplazar `COLISI?N → COLISIÓN`, `EMBARRANC? → EMBARRANCÓ`, `CA?DA → CAÍDA`, `ENCUNET? → ENCUNETADO` (validar con catálogo oficial).
3. **Definir catálogo canónico** de `tipo_eve` (p. ej., `COLISIÓN, ATROPELLO, DERRAPE, CHOQUE, VUELCO, EMBARRANCÓ, CAÍDA, ENCUNETADO, NO_CLASIFICADO`).
4. **Documentar el cambio de codificación temporal** (2014–2020 códigos, 2021 códigos con decimal, 2022 etiquetas con errores, 2023 etiquetas limpias) para interpretaciones históricas.

## Impacto en análisis posteriores
- Las comparaciones por tipo entre años requieren aplicar el **mapeo canónico**; de lo contrario, se mezclarán categorías (p. ej., `1` vs. `COLISIÓN`).
- Las métricas de tendencia por tipo (series 2014–2023) solo serán válidas tras **normalizar** y **corregir encoding**.


## **Inciso 4**

In [0]:
# INCISO 4 — Departamentos únicos por base (Hechos, Vehículos, Personas) y total
# Requiere: df_all, hechos_df, vehiculos_df, personas_df (definidos en Inciso 1)

from pyspark.sql import functions as F

def depto_summary(df, name):
    if "depto_ocu" not in df.columns:
        print(f"[WARN] {name}: no existe la columna 'depto_ocu'")
        return

    d = (
        df
        .withColumn("depto_str", F.upper(F.trim(F.col("depto_ocu").cast("string"))))
        .withColumn("is_numeric", F.col("depto_str").rlike(r"^[0-9]+(\.0+)?$"))
        .withColumn("depto_code", F.when(F.col("is_numeric"),
                                         F.regexp_extract(F.col("depto_str"), r"^[0-9]+", 0).cast("int")))
        .withColumn("depto_name", F.when(~F.col("is_numeric"), F.col("depto_str")))
    )

    total_rows = df.count()
    raw_distinct = d.select(F.col("depto_str")).where(F.col("depto_str").isNotNull()).agg(F.countDistinct("depto_str").alias("n")).first()["n"]
    code_distinct = d.select("depto_code").where(F.col("depto_code").isNotNull()).agg(F.countDistinct("depto_code").alias("n")).first()["n"]
    name_distinct = d.select("depto_name").where(F.col("depto_name").isNotNull()).agg(F.countDistinct("depto_name").alias("n")).first()["n"]

    print(f"\n=== {name} ===")
    print(f"Filas totales: {total_rows:,}")
    print(f"Distinct 'depto_ocu' (normalizado string): {raw_distinct}")
    print(f"Distinct CÓDIGOS numéricos:                {code_distinct}")
    print(f"Distinct NOMBRES/etiquetas:                 {name_distinct}")

    # Listados útiles (pueden ayudar para validar que sean 22 deptos u outliers)
    print("\nTop códigos (ordenados):")
    display(
        d.where(F.col("depto_code").isNotNull())
         .groupBy("depto_code").count().orderBy("depto_code")
    )

    print("Top nombres/etiquetas (alfabético):")
    display(
        d.where(F.col("depto_name").isNotNull())
         .groupBy("depto_name").count().orderBy("depto_name")
    )

# ---- Ejecutar por cada base lógica
depto_summary(hechos_df,    "Hechos")
depto_summary(vehiculos_df, "Vehículos")
depto_summary(personas_df,  "Personas")

# ---- Resumen TOTAL (sobre todo el unificado)
print("\n=== TOTAL (df_all) ===")
d_all = (
    df_all
    .withColumn("depto_str", F.upper(F.trim(F.col("depto_ocu").cast("string"))))
    .withColumn("is_numeric", F.col("depto_str").rlike(r"^[0-9]+(\.0+)?$"))
    .withColumn("depto_code", F.when(F.col("is_numeric"),
                                     F.regexp_extract(F.col("depto_str"), r"^[0-9]+", 0).cast("int")))
    .withColumn("depto_name", F.when(~F.col("is_numeric"), F.col("depto_str")))
)

raw_distinct_all  = d_all.select("depto_str").where(F.col("depto_str").isNotNull()).agg(F.countDistinct("depto_str").alias("n")).first()["n"]
code_distinct_all = d_all.select("depto_code").where(F.col("depto_code").isNotNull()).agg(F.countDistinct("depto_code").alias("n")).first()["n"]
name_distinct_all = d_all.select("depto_name").where(F.col("depto_name").isNotNull()).agg(F.countDistinct("depto_name").alias("n")).first()["n"]

print(f"Distinct 'depto_ocu' (string): {raw_distinct_all}")
print(f"Distinct CÓDIGOS:              {code_distinct_all}")
print(f"Distinct NOMBRES:              {name_distinct_all}")

print("\nListado total de códigos (ordenados):")
display(
    d_all.where(F.col("depto_code").isNotNull())
         .groupBy("depto_code").count().orderBy("depto_code")
)

print("Listado total de nombres (alfabético):")
display(
    d_all.where(F.col("depto_name").isNotNull())
         .groupBy("depto_name").count().orderBy("depto_name")
)



=== Hechos ===
Filas totales: 70,435
Distinct 'depto_ocu' (normalizado string): 72
Distinct CÓDIGOS numéricos:                22
Distinct NOMBRES/etiquetas:                 28

Top códigos (ordenados):


depto_code,count
1,20797
2,1240
3,1765
4,2193
5,4015
6,1838
7,1001
8,507
9,1862
10,1816


Top nombres/etiquetas (alfabético):


depto_name,count
ALTA VERAPAZ,711
BAJA VERAPAZ,290
CHIMALTENANGO,597
CHIQUIMULA,247
EL PROGRESO,320
ESCUINTLA,1625
GUATEMALA,6844
HUEHUETENANGO,244
IZABAL,477
JALAPA,245



=== Vehículos ===
Filas totales: 86,130
Distinct 'depto_ocu' (normalizado string): 72
Distinct CÓDIGOS numéricos:                22
Distinct NOMBRES/etiquetas:                 28

Top códigos (ordenados):


...

depto_name,count



=== TOTAL (df_all) ===
Distinct 'depto_ocu' (string): 72
Distinct CÓDIGOS:              22
Distinct NOMBRES:              28

Listado total de códigos (ordenados):


depto_code,count
1,26755
2,1707
3,2204
4,2804
5,5427
6,2502
7,1370
8,739
9,2892
10,2700


Listado total de nombres (alfabético):


depto_name,count
ALTA VERAPAZ,711
BAJA VERAPAZ,290
CHIMALTENANGO,597
CHIQUIMULA,247
EL PROGRESO,320
ESCUINTLA,1625
GUATEMALA,6844
HUEHUETENANGO,244
IZABAL,477
JALAPA,245


# Inciso 4 — Análisis de **departamentos únicos** (`depto_ocu`)

## 1) Resumen por subtabla
- **Hechos**  
  - Filas: **70,435**  
  - Distintos como *string*: **72**  
  - **Códigos** distintos: **22**  
  - **Nombres** distintos: **28**

- **Vehículos**  
  - Filas: **86,130**  
  - Distintos como *string*: **72**  
  - **Códigos** distintos: **22**  
  - **Nombres** distintos: **28**

- **Personas**  
  - Filas: **12,505**  
  - Distintos como *string*: **44**  
  - **Códigos** distintos: **22**  
  - **Nombres** distintos: **0** (esta subtabla no trae nombres; solo códigos)

- **Total (df_all)**  
  - Distintos como *string*: **72**  
  - **Códigos** distintos: **22**  
  - **Nombres** distintos: **28**

**Conclusión inmediata:** existen los **22 códigos** de departamento esperados (consistencia estructural), pero hay **inflación de etiquetas** (28 nombres) debido a variantes y problemas de codificación.

---

## 2) Patrones y problemas detectados
1. **Heterogeneidad de representación**
   - Convivencia de **códigos numéricos** (1…22) y **nombres** (e.g., *GUATEMALA*, *ESCUINTLA*).
   - La subtabla **Personas** utiliza **solo códigos**, lo que simplifica su estandarización.

2. **Variantes ortográficas y de acentuación**
   - Duplicados por acentos y/o mayúsculas:  
     - `PET?N` ↔ `PETÉN`  
     - `QUICH?` ↔ `QUICHÉ`  
     - `SACATEP?QUEZ` ↔ `SACATEPEQUEZ` (y variantes con acento)  
     - `SOLOL?` ↔ `SOLOLÁ`  
     - `TOTONICAP?N` ↔ `TOTONICAPÁN`
   - Esto explica por qué los **nombres** suman **28** en lugar de 22.

3. **Distribución por código (tendencia)**
   - Los **códigos 1, 5, 16** (y cercanos) presentan los conteos más altos, consistentes con departamentos de alta carga de hechos/vehículos (p. ej., código **1** es el más frecuente en todas las vistas).  
   - El **nombre GUATEMALA** aparece con la mayor frecuencia entre etiquetas, coherente con el patrón por código.

> Nota: los conteos por código **no son comparables sumando subtablas** porque las subtablas son **subconjuntos solapados** del mismo archivo unificado (no disjuntos).

---

## 3) Implicaciones para uniones y agregaciones
- Dado que **Personas** carece de nombres, la integración entre subtablas debe **pivotar en los códigos** de `depto_ocu`.
- Las agregaciones por departamento pueden verse **sesgadas** si se mezclan **nombres con variantes**; conviene **normalizar a código** y, en todo caso, derivar el nombre canónico a partir de un **diccionario maestro**.


## 5) Qué significa para el resto del laboratorio
- Los incisos que pidan métricas por **departamento** deben realizarse sobre **`depto_code`**.  
- Cuando se solicite graficar o reportar por nombre, se recomienda **proyectar el nombre canónico** desde el diccionario (evitando variaciones y errores de codificación).


## **Antes de inciso 5, normalizamos**

In [0]:
# HOTFIX NORMALIZACIÓN — tolerante a "999.0" y similares
from pyspark.sql import functions as F

# === Diccionarios de apoyo (igual que antes) ===
MONTHS_MAP = {
    "ENERO": 1, "FEBRERO": 2, "MARZO": 3, "ABRIL": 4, "MAYO": 5, "JUNIO": 6,
    "JULIO": 7, "AGOSTO": 8, "SEPTIEMBRE": 9, "SETIEMBRE": 9,
    "OCTUBRE": 10, "NOVIEMBRE": 11, "DICIEMBRE": 12
}
DEPTO_NAME_FIX = {
    "PET?N": "PETÉN", "QUICH?": "QUICHÉ", "SACATEP?QUEZ": "SACATEPÉQUEZ",
    "SACATEPEQUEZ": "SACATEPÉQUEZ", "SUCHITEP?QUEZ": "SUCHITEPÉQUEZ",
    "SUCHITEPEQUEZ": "SUCHITEPÉQUEZ", "SOLOL?": "SOLOLÁ", "TOTONICAP?N": "TOTONICAPÁN",
    "PETÉN": "PETÉN", "QUICHÉ": "QUICHÉ", "SACATEPÉQUEZ": "SACATEPÉQUEZ",
    "SUCHITEPÉQUEZ": "SUCHITEPÉQUEZ", "SOLOLÁ": "SOLOLÁ", "TOTONICAPÁN": "TOTONICAPÁN",
}
TIPO_EVE_FIX = {
    "COLISI?N": "COLISIÓN", "COLISION": "COLISIÓN",
    "EMBARRANC?": "EMBARRANCÓ", "EMBARRANCO": "EMBARRANCÓ",
    "CA?DA": "CAÍDA", "ENCUNET?": "ENCUNETADO",
}
TIPO_EVE_CODE_TO_LABEL = {
    1: "COLISIÓN", 5: "ATROPELLO", 2: "DERRAPE", 6: "CHOQUE",
    4: "VUELCO", 7: "EMBARRANCÓ", 8: "CAÍDA/ENCUNETADO", 99: "NO_CLASIFICADO",
    3: "OTRO_3",
}
SENTINELS_STR = {"", " ", "NA", "N/A", "NULL", "IGNORADO", "IGNORADA", "IGNORADO/A", "NO APLICA", "NO_APLICA"}
SENTINELS_INT = {99, 999, 9999}

# === Helpers (100% expresiones Spark) ===
def _norm_str(colname):
    return F.upper(F.trim(F.col(colname).cast("string")))

def _map_from_dict(expr_str, mapping):
    pairs = []
    for k, v in mapping.items():
        pairs += [F.lit(k), F.lit(v)]
    return F.element_at(F.create_map(*pairs), expr_str)

def _strip_dotzero_to_int(colname):
    """
    Convierte "15" o "15.0" -> 15 (INT). Otras cosas -> NULL.
    """
    s = _norm_str(colname)
    return F.when(
        s.rlike(r"^\d+(\.0+)?$"),
        F.regexp_extract(s, r"^\d+", 0).cast("int")
    )

def _null_sentinel_any(colname):
    """
    Devuelve NULL si:
      - es un sentinel textual (NA, N/A, etc.), o
      - es un número (con o sin '.0') y su parte entera está en {99, 999, 9999}.
    En caso contrario devuelve el valor original.
    """
    s = _norm_str(colname)
    int_clean = F.when(
        s.rlike(r"^\d+(\.0+)?$"),
        F.regexp_extract(s, r"^\d+", 0).cast("int")
    )
    is_sentinel_text = s.isin(*list(SENTINELS_STR))
    is_sentinel_int  = int_clean.isin(*list(SENTINELS_INT))
    return F.when(is_sentinel_text | is_sentinel_int, F.lit(None)).otherwise(F.col(colname))

def _year_from_any(colname):
    # extrae 4 dígitos como año y los castea a INT
    return F.regexp_extract(_norm_str(colname), r"(\d{4})", 1).cast("int")

def _month_from_any(colname):
    s = _norm_str(colname)
    # mapear nombres → número, o tomar la parte entera si es "12" / "12.0"
    return F.coalesce(
        _map_from_dict(s, MONTHS_MAP),
        _strip_dotzero_to_int(colname)
    )

def _hour_from_any(colname):
    s = _norm_str(colname)
    # Ignorada → NULL; si es número (o número.0), tomar entero
    return F.when(s.isin("IGNORADA", "IGNORADO"), F.lit(None)).otherwise(_strip_dotzero_to_int(colname))

# === Construcción segura del df normalizado ===
base = df_all

df_norm = (
    base
    # Tiempo
    .withColumn("anio", _year_from_any("anio_ocu"))
    .withColumn("mes",  _month_from_any("mes_ocu"))
    .withColumn("dia",  _strip_dotzero_to_int("dia_ocu"))
    .withColumn("hora", _hour_from_any("hora_ocu"))

    # Ubicación
    .withColumn("depto_raw", _norm_str("depto_ocu"))
    .withColumn("depto_is_num", F.col("depto_raw").rlike(r"^[0-9]+(\.0+)?$"))
    .withColumn("depto_code", F.when(F.col("depto_is_num"),
                                     F.regexp_extract(F.col("depto_raw"), r"^\d+", 0).cast("int")))
    .withColumn("depto_name_raw", F.when(~F.col("depto_is_num"), F.col("depto_raw")))
    .withColumn("depto_name_std", F.coalesce(_map_from_dict(F.col("depto_name_raw"), DEPTO_NAME_FIX),
                                             F.col("depto_name_raw")))

    # Tipo de evento
    .withColumn("tipo_eve_raw", _norm_str("tipo_eve"))
    .withColumn("tipo_eve_fix", F.coalesce(_map_from_dict(F.col("tipo_eve_raw"), TIPO_EVE_FIX),
                                           F.col("tipo_eve_raw")))
    .withColumn("tipo_eve_code", F.when(F.col("tipo_eve_fix").rlike(r"^\d+(\.0+)?$"),
                                        F.regexp_extract(F.col("tipo_eve_fix"), r"^\d+", 0).cast("int")))
    .withColumn("tipo_eve_std", F.coalesce(_map_from_dict(F.col("tipo_eve_code"), TIPO_EVE_CODE_TO_LABEL),
                                           F.col("tipo_eve_fix")))

    # Limpieza de sentinelas (aplica a varios campos)
    .withColumn("modelo_veh_std",      _null_sentinel_any("modelo_veh"))
    .withColumn("g_modelo_veh_std",    _null_sentinel_any("g_modelo_veh"))
    .withColumn("color_veh_std",       _null_sentinel_any("color_veh"))
    .withColumn("marca_veh_std",       _null_sentinel_any("marca_veh"))

    # Edades (entero limpio, sentinelas → NULL)
    .withColumn("edad_pil_int_raw", _strip_dotzero_to_int("edad_pil"))
    .withColumn("edad_pil_int", F.when(F.col("edad_pil_int_raw").isin(*list(SENTINELS_INT)), None)
                                  .otherwise(F.col("edad_pil_int_raw")))
    .drop("edad_pil_int_raw")

    .withColumn("edad_con_int_raw", _strip_dotzero_to_int("edad_con"))
    .withColumn("edad_con_int", F.when(F.col("edad_con_int_raw").isin(*list(SENTINELS_INT)), None)
                                  .otherwise(F.col("edad_con_int_raw")))
    .drop("edad_con_int_raw")

    .withColumn("edad_per_int_raw", _strip_dotzero_to_int("edad_per"))
    .withColumn("edad_per_int", F.when(F.col("edad_per_int_raw").isin(*list(SENTINELS_INT)), None)
                                  .otherwise(F.col("edad_per_int_raw")))
    .drop("edad_per_int_raw")
)

# === Reporte rápido ===
print("=== RESUMEN NORMALIZACIÓN (hotfix) ===")
print("Filas:", df_norm.count())
print("Años (min, max):", df_norm.select(F.min("anio"), F.max("anio")).first())
print("Mes (min, max):",  df_norm.select(F.min("mes"),  F.max("mes")).first())
print("Hora (min, max):", df_norm.select(F.min("hora"), F.max("hora")).first())
print("Depto: códigos distintos:",
      df_norm.select("depto_code").where(F.col("depto_code").isNotNull()).agg(F.countDistinct("depto_code")).first()[0])
print("Depto: nombres distintos (std):",
      df_norm.select("depto_name_std").where(F.col("depto_name_std").isNotNull()).agg(F.countDistinct("depto_name_std")).first()[0])
print("Tipo_eve: códigos distintos:",
      df_norm.select("tipo_eve_code").where(F.col("tipo_eve_code").isNotNull()).agg(F.countDistinct("tipo_eve_code")).first()[0])
print("Tipo_eve: etiquetas distintas (std):",
      df_norm.select("tipo_eve_std").where(F.col("tipo_eve_std").isNotNull()).agg(F.countDistinct("tipo_eve_std")).first()[0])

# Vista para siguientes incisos
df_norm.createOrReplaceTempView("accidentes_norm")

# (Opcional) muestra rápida
display(
    df_norm.select("anio","mes","dia","hora","depto_code","depto_name_std",
                   "tipo_eve_code","tipo_eve_std","modelo_veh_std","color_veh_std")
           .limit(20)
)


=== RESUMEN NORMALIZACIÓN (hotfix) ===
Filas: 89317
Años (min, max): Row(min(anio)=2009, max(anio)=2023)
Mes (min, max): Row(min(mes)=1, max(mes)=12)
Hora (min, max): Row(min(hora)=0, max(hora)=99)
Depto: códigos distintos: 22
Depto: nombres distintos (std): 22
Tipo_eve: códigos distintos: 9
Tipo_eve: etiquetas distintas (std): 11


anio,mes,dia,hora,depto_code,depto_name_std,tipo_eve_code,tipo_eve_std,modelo_veh_std,color_veh_std
2009,5,13,1,14,null,null,null,1988.0,2.0
2009,7,1,15,22,null,null,null,null,4.0
2009,3,1,17,17,null,null,null,null,null
2009,2,26,18,21,null,null,null,null,1.0
2009,3,1,15,10,null,null,null,2007.0,1.0
2009,4,5,14,18,null,null,null,null,1.0
2009,3,26,8,12,null,null,null,null,6.0
2009,6,27,11,15,null,null,null,null,1.0
2009,4,14,14,15,null,null,null,null,2.0
2009,3,25,9,12,null,null,null,null,3.0


## **Inciso 5**

In [0]:
# INCISO 5 — Total de accidentes por año y departamento (gráfico de barras con display)
# Requiere: df_norm (creado en la normalización). Si no existe, vuelve a ejecutar el Inciso 1 + normalización.

from pyspark.sql import functions as F

# 1) Filtrar filas que representan hechos/accidentes
hechos = df_norm.where(
    F.col("tipo_eve_std").isNotNull() | F.col("tipo_eve_code").isNotNull()
)

# 2) Agregar: accidentes por año y departamento
acc_x_anio_depto = (
    hechos
    .where(F.col("anio").isNotNull() & F.col("depto_code").isNotNull())
    .groupBy("anio", "depto_code", "depto_name_std")
    .agg(F.count(F.lit(1)).alias("accidentes"))
    .orderBy("anio", "depto_code")
)

# 3) Mostrar tabla y graficar con display:
#    En la UI de Databricks:
#      - haz click en "Plot Options"
#      - Chart type: "Bar"
#      - Keys: anio
#      - Series: depto_name_std (o depto_code si prefieres)
#      - Values: accidentes
display(acc_x_anio_depto)

# 4) (Opcional) Top 10 departamentos por accidentes acumulados (para gráficos más legibles)
top10 = (
    acc_x_anio_depto
    .groupBy("depto_code", "depto_name_std")
    .agg(F.sum("accidentes").alias("acc_total"))
    .orderBy(F.desc("acc_total"))
    .limit(10)
    .select("depto_code")
)

acc_x_anio_depto_top10 = acc_x_anio_depto.join(top10, on="depto_code", how="inner")

#    Graficar de igual forma que arriba (Bar: Keys=anio, Series=depto_name_std, Values=accidentes)
display(acc_x_anio_depto_top10)

# 5) (Opcional) Foco en un año específico (p. ej., 2023) para ver ranking por depto
acc_2023 = (
    acc_x_anio_depto
    .where(F.col("anio") == 2023)
    .orderBy(F.desc("accidentes"))
)

#    Para este, en "Plot Options":
#      - Chart type: "Bar"
#      - Keys: depto_name_std
#      - Values: accidentes
display(acc_2023)

# 6) (Opcional) Guardar vistas temporales para reutilizar en incisos siguientes
acc_x_anio_depto.createOrReplaceTempView("accidentes_por_anio_depto")
acc_x_anio_depto_top10.createOrReplaceTempView("accidentes_por_anio_depto_top10")


anio,depto_code,depto_name_std,accidentes
2014,1,null,1912
2014,2,null,127
2014,3,null,150
2014,4,null,228
2014,5,null,444
2014,6,null,242
2014,7,null,109
2014,8,null,82
2014,9,null,255
2014,10,null,223


depto_code,anio,depto_name_std,accidentes
16,2015,null,315
18,2015,null,285
10,2014,null,223
16,2014,null,223
4,2015,null,263
17,2014,null,219
1,2014,null,1912
9,2015,null,316
10,2015,null,243
9,2014,null,255


anio,depto_code,depto_name_std,accidentes


## **Inciso 6**

In [0]:
# INCISO 6 — Día de la semana con más accidentes en 2024 (o año más reciente disponible)
# Requiere: df_norm del paso de normalización.

from pyspark.sql import functions as F

# 1) Tomar la "tabla de hechos"
hechos = df_norm.where(F.col("anio").isNotNull() & (F.col("tipo_eve_std").isNotNull() | F.col("tipo_eve_code").isNotNull()))

# 2) Normalizar día de la semana:
#    - Acepta numéricos "1", "1.0"… y texto "Lunes/Martes/Miércoles/..."
def _norm_str(c): 
    return F.upper(F.trim(F.col(c).cast("string")))

TEXT2NUM = {
    "LUNES": 1, "MARTES": 2, "MIERCOLES": 3, "MIÉRCOLES": 3,
    "JUEVES": 4, "VIERNES": 5, "SABADO": 6, "SÁBADO": 6, "DOMINGO": 7
}
NUM2TEXT = {1:"LUNES", 2:"MARTES", 3:"MIÉRCOLES", 4:"JUEVES", 5:"VIERNES", 6:"SÁBADO", 7:"DOMINGO"}

dia_txt = _norm_str("dia_sem_ocu")
dia_num_from_text = F.element_at(F.create_map(*sum([[F.lit(k), F.lit(v)] for k,v in TEXT2NUM.items()], [])), dia_txt)
dia_num_from_num  = F.when(dia_txt.rlike(r"^\d+(\.0+)?$"), F.regexp_extract(dia_txt, r"^\d+", 0).cast("int"))
dia_num = F.coalesce(dia_num_from_num, dia_num_from_text)

hechos_dow = (
    hechos
    .withColumn("dow_num", dia_num)
    .withColumn("dow_label", F.element_at(F.create_map(*sum([[F.lit(k), F.lit(v)] for k,v in NUM2TEXT.items()], [])), F.col("dow_num")))
    .where(F.col("dow_num").between(1,7))  # mantener solo 1..7
)

# 3) Elegir año objetivo: 2024 si existe, si no el más reciente disponible
exists_2024 = hechos_dow.where(F.col("anio")==2024).limit(1).count() > 0
max_year = hechos_dow.agg(F.max("anio").alias("maxy")).first()["maxy"]
target_year = 2024 if exists_2024 else max_year
print(f"[INFO] Año usado para el inciso 6: {target_year}" + (" (2024 no disponible)" if not exists_2024 else ""))

# 4) Agregar y ordenar (para graficar en columnas)
acc_por_dow = (
    hechos_dow
    .where(F.col("anio")==target_year)
    .groupBy("dow_num", "dow_label")
    .agg(F.count(F.lit(1)).alias("accidentes"))
    .orderBy("dow_num")
)

display(acc_por_dow)  # Plot Options: Chart type = Column, Keys = dow_label, Values = accidentes

# 5) Día con más accidentes (imprimir)
top = acc_por_dow.orderBy(F.desc("accidentes")).limit(1).collect()
if top:
    print(f"→ Máximo en {target_year}: {top[0]['dow_label']} ({top[0]['accidentes']} accidentes)")

# 6) Vista temporal opcional para reutilizar
acc_por_dow.createOrReplaceTempView("accidentes_por_dow")


[INFO] Año usado para el inciso 6: 2023 (2024 no disponible)


dow_num,dow_label,accidentes
1,LUNES,1053
2,MARTES,933
3,MIÉRCOLES,924
4,JUEVES,956
5,VIERNES,1175
6,SÁBADO,1557
7,DOMINGO,1620


→ Máximo en 2023: DOMINGO (1620 accidentes)


# Inciso 6 — Análisis por día de la semana (razones de los picos)

## Hallazgos principales
- **Máximos:** **Domingo (1,620)** y **Sábado (1,557)**.
- **Intermedio-alto:** **Viernes (1,175)**.
- **Mínimos relativos:** **Lunes (1,053)**, **Martes (933)**, **Miércoles (924)**, **Jueves (956)**.
- Interpretación: el volumen de accidentes **aumenta hacia el fin de semana**, alcanzando su punto más alto el **domingo**.

## Posibles razones (explicativas, no causales)
1. **Mayor movilidad recreativa y social**  
   Los fines de semana concentran viajes por ocio, visitas familiares, salidas nocturnas y actividades deportivas/culturales. Esto **eleva la exposición** (vehículos-km recorridos), aumentando la probabilidad de incidentes.

2. **Conducción nocturna y fatiga**  
   Sábado y madrugada de domingo suelen tener **más conducción nocturna**, con **fatiga acumulada** y menor visibilidad; ambos factores se asocian a mayor riesgo de siniestros.

3. **Consumo de alcohol en ocio nocturno**  
   Las noches de fin de semana presentan mayor prevalencia de **conducción bajo efectos del alcohol**, factor de riesgo reconocido en choques/colisiones.

4. **Cambios de patrón de tráfico**  
   El viernes concentra **congestión de salida laboral** y traslados interurbanos; sábado y domingo muestran **velocidades más altas** por menor tráfico, lo que puede **incrementar la severidad** y probabilidad de pérdida de control (e.g., derrapes, vuelcos).

5. **Distribución modal y peatonal**  
   En fines de semana se observan más **peatones y ciclistas** en áreas recreativas; si existen cruces inseguros o infraestructura limitada, **atropellos** pueden aumentar.

## Implicaciones prácticas
- **Enfoque de fiscalización**: reforzar **controles nocturnos de fin de semana** (alcoholimetría, velocidad) y **puntos críticos** por atropellos.  
- **Intervenciones de infraestructura**: mejorar **iluminación, señalización y pasos seguros** en corredores recreativos y accesos urbanos con alta siniestralidad en fines de semana.  
- **Campañas de prevención**: mensajes dirigidos a conducción responsable los **viernes por la tarde/noche** y durante **sábado–domingo**, cuando el riesgo agregado es mayor.


In [0]:
# INCISO 7 — Distribución de accidentes por HORA del día en el municipio de Guatemala
# Requiere: df_norm (del paso de normalización)

from pyspark.sql import functions as F

# 1) Filtrar a "hechos" válidos y derivar código de municipio (entero robusto)
hechos = (
    df_norm
    .where(F.col("anio").isNotNull() & (F.col("tipo_eve_std").isNotNull() | F.col("tipo_eve_code").isNotNull()))
    .withColumn("muni_str",  F.upper(F.trim(F.col("muni_ocu").cast("string"))))
    .withColumn("muni_code", F.when(F.col("muni_str").rlike(r"^\d+(\.0+)?$"),
                                    F.regexp_extract(F.col("muni_str"), r"^\d+", 0).cast("int")))
    .withColumn("hora_clean", F.when(F.col("hora").between(0,23), F.col("hora")))
)

# 2) Municipio de Guatemala: depto_code == 1 y muni_code == 101 (esquema depto*100 + muni)
hechos_gua = hechos.where(
    (F.col("depto_code") == 1) & (F.col("muni_code") == 101) & F.col("hora_clean").isNotNull()
)

# 3) Conteo por hora (para tabla)
dist_hora = (
    hechos_gua
    .groupBy("hora_clean")
    .agg(F.count(F.lit(1)).alias("accidentes"))
    .orderBy("hora_clean")
)

print("[INFO] Municipio de Guatemala — filas consideradas:", hechos_gua.count())
display(dist_hora)

# 4) Histograma (Databricks)
#   En Plot Options:
#     - Chart type: Histogram
#     - Values: hora_clean
#     - (Opcional) Number of bins: 24   (o Bin size: 1)
display(hechos_gua.select("hora_clean"))

# 5) (Opcional) Vista para reutilizar
dist_hora.createOrReplaceTempView("accidentes_guatemala_por_hora")

# 6) (Opcional) Top 3 horas con más accidentes (impresión rápida)
top3 = dist_hora.orderBy(F.desc("accidentes")).limit(3).collect()
if top3:
    print("Top 3 horas (Municipio de Guatemala):")
    for r in top3:
        print(f"  - {int(r['hora_clean']):02d}:00 → {r['accidentes']} accidentes")


[INFO] Municipio de Guatemala — filas consideradas: 11620


hora_clean,accidentes
0,501
1,504
2,389
3,333
4,258
5,293
6,321
7,415
8,369
9,401


hora_clean
3
3
19
17
18
9
20
17
1
1


Top 3 horas (Municipio de Guatemala):
  - 21:00 → 724 accidentes
  - 23:00 → 696 accidentes
  - 22:00 → 691 accidentes


## Distribución horaria de accidentes — Municipio de Guatemala

**Base analizada:** 11,620 registros con hora válida (0–23).  
**Top 3 horas:** 21:00 (724), 23:00 (696), 22:00 (691).

### 1) Patrón general
- Se observa un **clúster vespertino-nocturno** muy marcado entre **17:00 y 23:00**, con un máximo en **21:00**.
- Las horas de **madrugada (02:00–05:00)** presentan los **mínimos** (p. ej., 04:00 = 258), mientras que la **mañana** muestra un **hombro moderado** (07:00–11:00).

| Hora | Accidentes |
|---:|---:|
| 21 | **724** |
| 23 | **696** |
| 22 | **691** |
| 19 | 689 |
| 20 | 686 |
| 18 | 606 |
| 17 | 592 |
| 15 | 531 |
| 00 | 501 |
| 12 | 466 |
| 10 | 463 |
| 14 | 455 |
| 09 | 401 |
| 08 | 369 |
| 13 | 382 |
| 02 | 389 |
| 07 | 415 |
| 11 | 406 |
| 16 | 449 |
| 01 | 504 |
| 03 | 333 |
| 05 | 293 |
| 06 | 321 |
| 04 | **258** |

> Nota: 21:00 concentra ~**6.2%** del total (724/11,620).

### 2) Posibles razones del pico vespertino-nocturno
- **Alto volumen vehicular** post-jornada laboral (17:00–20:00) → más exposición y **congestión**, que incrementan la probabilidad de siniestros.
- **Condiciones de visibilidad** y **fatiga**: al anochecer y noche (18:00–23:00) disminuye la visibilidad y aumenta el cansancio acumulado del día.
- **Movilidad social/nocturna** (20:00–23:00): desplazamientos a actividades recreativas y retornos a casa; se combinan **prisas**, **distráctores** y, en algunos casos, **consumo de alcohol**.
- **Mezcla modal más riesgosa**: microbuses, motocicletas y peatones interactúan en franjas de alta demanda nocturna, elevando el riesgo de colisión y atropello.

### 3) ¿Por qué hay un “hombro” matutino y mínimos de madrugada?
- **Mañana (07:00–11:00)**: inicio de actividades (trabajo/estudio) y **picos de entrada** generan un aumento moderado (p. ej., 10:00 = 463).
- **Madrugada (02:00–05:00)**: **bajo flujo** reduce la frecuencia absoluta de accidentes; aun así, 00:00–01:00 se mantienen relativamente altos (501–504), consistente con **salidas nocturnas** y retornos tardíos.
- **Transiciones de luz** (amanecer/atardecer) pueden añadir **riesgo perceptivo** (deslumbramiento, contraste).

### 4) Implicaciones operativas
- **Fiscalización y control** focalizados en **19:00–23:00** (alcoholímetro, velocidad, distracciones).
- **Gestión de semáforos y señalización**: priorizar corredores críticos en **hora pico vespertina**.
- **Iluminación y visibilidad**: mantenimiento de luminarias, pintura de demarcaciones y cruces peatonales donde la demanda nocturna es alta.
- **Campañas de seguridad vial** temporizadas para el **retorno** (after-work/ocio).

### 5) Consideraciones y siguientes pasos
- Esta distribución corresponde al **municipio de Guatemala** y a **todos los años disponibles** en la muestra (no está restringida a un solo año).
- Para enriquecer el diagnóstico:
  - Desagregar por **día de semana** (fin de semana vs. laboral) y **tipo de accidente**.
  - Controlar por **estacionalidad** (mes) y **clima** si la fuente lo permite.
  - Analizar la participación de **motocicletas/peatones** en las horas pico.

